In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
!pip install -q -U keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.5 MB/s eta 0:00:00


In [3]:
import keras_tuner as kt

In [4]:
(img_train,label_train),(img_test,label_test)=keras.datasets.fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [5]:
img_train = img_train.astype('float32')/255.0
img_test = img_test.astype('float32')/255.0

In [8]:
#zdefiniowanie modelu z hiperparametrami
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28,28)))

  hp_units = hp.Int('units',min_value=32,max_value=512,step=32)
  model.add(keras.layers.Dense(units=hp_units,activation='relu'))

  model.add(keras.layers.Dense(10))

  hp_learning_rate = hp.Choice('learning_rate',values=[1.0,1e-2,1e-3,1e-4,1e-5])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [9]:
#utowrzenie wystąpienia tunera i hypertuning
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [10]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5)

In [ ]:
#wyszukiwanie hiperparametrów
tuner.search(img_train,label_train,epochs=50,validation_split=0.2,callbacks=[stop_early])
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_hps)

print(f"""
        Wyszukiwanie hiperparametrów zakończone.
        Najlepsze hiperparametry:
        Liczba neuronów: {best_hps.get('units')}
        Szybkość uczenia: {best_hps.get('learning_rate')}
""")

Trial 9 Complete [00h 00m 12s]
val_accuracy: 0.687749981880188

Best val_accuracy So Far: 0.8646666407585144
Total elapsed time: 00h 03m 43s

Search: Running Trial #10

Value             |Best Value So Far |Hyperparameter
192               |352               |units
1                 |0.001             |learning_rate
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.1074 - loss: 144.9654 - val_accuracy: 0.1004 - val_loss: 2.3920
Epoch 2/2
 304/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.0952 - loss: 2.4175